In [3]:
import cv2
import easyocr
import json
from PIL import Image, ImageDraw
import re

# EasyOCR reader 객체 생성 (한글 및 영어 지원)
reader = easyocr.Reader(['ko', 'en'], gpu=False)

def process_image(img):
    """
    이미지를 읽고 텍스트를 추출하여, 가로로 긴 박스를 그립니다.
    """
    # 이미지 읽기
    image = cv2.imread(img)
    if image is None:
        raise Exception(f"이미지를 읽을 수 없습니다: {img}")
    
    # 이미지에서 텍스트 추출
    results = reader.readtext(image)
    
    # 추출된 텍스트를 저장할 리스트
    extracted_text = []
    
    # 파란색 상자를 그리기 위한 PIL 이미지로 변환
    img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img)
    
    # 각 텍스트 주위에 상자를 그리고 텍스트를 저장
    for detection in results:
        bbox = detection[0]
        text = detection[1]
        extracted_text.append(text)
        
        # x, y 좌표를 이용해 가로형 박스 설정
        x_min = min(bbox[0][0], bbox[1][0])
        x_max = max(bbox[2][0], bbox[3][0])
        y_min = min(bbox[0][1], bbox[1][1], bbox[2][1], bbox[3][1])
        y_max = max(bbox[0][1], bbox[1][1], bbox[2][1], bbox[3][1])
        
        # 일정한 높이의 가로형 박스를 생성
        box_height = 30  # 원하는 박스 높이로 조절 가능
        y_center = (y_min + y_max) / 2
        y_min = int(y_center - box_height / 2)
        y_max = int(y_center + box_height / 2)
        
        # 상자와 텍스트 색상 설정
        box_color = "blue"
        text_color = "blue"
        
        # 상자 그리기
        draw.rectangle([x_min, y_min, x_max, y_max], outline=box_color, width=2)
        draw.text((x_min, y_min - 20), text, fill=text_color)
        
    # 정보 추출 함수들
    def extract_business_numbers(text_list):
        business_number_pattern = re.compile(r'\b\d{3}-?\d{2}-?\d{5}\b')
        return [match for text in text_list for match in business_number_pattern.findall(text)]
        
    def extract_store_names(text_list):
        # 오타를 포함한 키워드 패턴
        store_name_pattern = re.compile(r'(매장명|상호명|회사명|업체명|가맹점명|가맣점명|[상싱성][호오]|[회훼]사)\s*[:;：]?\s*([^\s)]+(?:\s*\S*)*?[점]\s*?\S*)')
        return [match[1] for text in text_list for match in store_name_pattern.findall(text)]
    
    def is_valid_date(date_str):
        """날짜 형식이 유효한지 검사하는 함수"""
        parts = re.split(r'[-/.]', date_str)  # '-', '/', '.' 구분자로 사용
        if len(parts) == 3:
            # YYYY-MM-DD 형식 체크
            if len(parts[0]) == 4 and len(parts[1]) == 2 and len(parts[2]) == 2:
                year, month, day = map(int, parts)
                return 2000 <= year <= 2100 and 1 <= month <= 12 and 1 <= day <= 31
            # DD/MM/YYYY 형식 체크
            elif len(parts[0]) == 2 and len(parts[1]) == 2 and len(parts[2]) == 4:
                day, month, year = map(int, parts)
                return 2000 <= year <= 2100 and 1 <= month <= 12 and 1 <= day <= 31
            # MM-DD-YY 형식 체크
            elif len(parts[0]) == 2 and len(parts[1]) == 2 and len(parts[2]) == 2:
                month, day, year = map(int, parts)
                year += 2000 if year < 100 else 0
                return 2000 <= year <= 2100 and 1 <= month <= 12 and 1 <= day <= 31
        return False

    def extract_transaction_date(json_data):
        # 다양한 날짜 형식을 허용하는 정규식
        date_pattern = re.compile(
            r'([거기][래레][일닐]|[결겔]제[일닐]|거[래레]일시|[결겔]제날짜|날짜|일자)?\s*[:;：]?\s*'
            r'(\d{4}[-/.]\d{2}[-/.]\d{2}|\d{2}[-/.]\d{2}[-/.]\d{4}|\d{2}[-/.]\d{2}[-/.]\d{2})'  # YYYY-MM-DD, DD/MM/YYYY, MM-DD-YY 형식
        )
    
        extracted_dates = []
    
        for text in json_data:  # json_data가 문자열 목록이라고 가정
            matches = date_pattern.findall(text)
    
            for match in matches:
                date_part = match[1]  # 날짜 부분만 추출
    
                # 유효한 날짜만 추가
                if is_valid_date(date_part):
                    extracted_dates.append(date_part)
    
        return extracted_dates



   
    
    # def extract_total_price(text_list):
    #     total_price_pattern = re.compile(
    #         r'(합계|총\s*금액|Total|Subtotal|거래금액)\s*[:;：]?\s*([₩$€]?\s*\d{1,3}(?:,\d{3})*(?:\.\d{2})?)\s*원?'
    #     )
    #     extracted_prices = []

    #     for text in text_list:
    #         matches = total_price_pattern.findall(text)

    #         for match in matches:
    #             price_with_unit = match[1]  # 가격 부분
    #             # "원"을 제거하고 float로 변환
    #             price = price_with_unit.replace(',', '').replace('₩', '').replace('$', '').replace('€', '').strip()
    #             extracted_prices.append(float(price))

    #     return max(extracted_prices, default=0) if extracted_prices else None

    # 최대 금액 추출
    def extract_max_number(text_list):
        all_numbers = []

        for text in text_list:
            # 모든 숫자 추출 (천 단위 쉼표 포함)
            numbers = re.findall(r'\d{1,3}(?:,\d{3})*', text)
            numbers = [int(num.replace(',', '')) for num in numbers]  # 쉼표 제거 후 정수형으로 변환
            all_numbers.extend(numbers)

        return max(all_numbers, default=0) if all_numbers else None

    info_dict = {
        "사업자번호": extract_business_numbers(extracted_text),
        "가맹점명": extract_store_names(extracted_text),
        "거래일시": extract_transaction_date(extracted_text),
        "금액": extract_max_number(extracted_text) 
    }
    
    return info_dict

# 이미지 처리 및 정보 추출
a = process_image('bill9.jpeg')
print(json.dumps(a, ensure_ascii=False, indent=4))


Using CPU. Note: This module is much faster with a GPU.


{
    "사업자번호": [
        "832-13-01366"
    ],
    "가맹점명": [],
    "거래일시": [
        "2024-10-25"
    ],
    "금액": 141000
}
